# Introduction to ExoCTK
Authors: Jules Fowler, Joe Filippazzo, Matthew Bourque, Kevin Stevenson

The Exoplanet Characterization Tool Kit (ExoCTK) is an open source Python 3.5+ package that serves as a one-stop-shop for observation planning, data reduction, and scientific analysis of transiting exoplanet systems.

In this notebook, we will cover the installation of the software as well as an end-to-end example that demonstrates the core functionalities of each tool provided by the ExoCTK package.

# Installation
You must first have a working installation of ``anaconda`` or ``miniconda`` for Python 3.  If you do not yet have this on your system, you can visit the following links for download and installation instructions:

- [Anaconda](https://www.anaconda.com/download/)
- [Miniconda](https://conda.io/en/latest/miniconda.html)

To obtain the latest `exoctk` package with the necessary environment files, you should clone the repository directly from GitHub:

```
git clone https://github.com/ExoCTK/exoctk.git
cd exoctk
```

To obtain the `exoctk` data, visit the shared [ExoCTK DropBox](https://www.dropbox.com/sh/p9c49urdbkmn5rz/AACFIjeAphOPqcCLtqd1v_W9a?dl=0), download the folders, and unzip them into a directory called `exoctk_data/`.  You may need to download  `modelgrid` seperately because it's the largest of the bunch and some computers have trouble with large .zip files.  Next, export an environment variable for `EXOCTK_DATA`. For Mac OS/Linux, add the line

```
export EXOCTK_DATA='/path/to/your/unzipped/directory/exoctk_data/'
```

to your `.bashrc` or `.bash_profile`. Don't forget to execute your `.bashrc` or `.bash_profile` to implement the changes in your terminal.

```
source ~/.bash_profile
```

For Windows, add an evironment variable using System Utility. 


You can install the ExoCTK `conda` environment via the `env/environment-<PYTHON_VERSION>.yml` files (relative to the parent directory of where the `exoctk` repository was installed).  Note that there are separate environment files for each version of `python` that `exoctk` supports.  For this demo, we will be using version 3.6. Ensure that `conda` is up to date, create the `exoctk` environment with Python version 3.6, and activate it with:

```
conda update conda
conda activate base
conda env create -f env/environment-3.6.yml
conda activate exoctk-3.6
conda install ipykernel
python -m ipykernel install --user --name exoctk-3.6 --display-name "exoctk-3.6"
```

Finally, install `exoctk` by running the `exoctk` setup script:

```
python setup.py install
```
# Kernel
Before getting started, make sure that you are using the `exoctk-3.6` kernel be selecting it via `Kernel` -> `Change kernel`.

# Imports
Here are a few basic imports we'll need later on...

In [ ]:
import os
import numpy as np
from bokeh.io import output_notebook
from bokeh.plotting import show
output_notebook()

# Ask a compelling science question
Successful proposals require a well-posed science question with testable hypothesis.  For this end-to-end demonstration, let's ask a simple question: "Where's the methane?"  More specifically, we want to know if planets with Teq < 1000 have methane in their atmospheres, as predicted by chemical equilibrium models.

# Find the best target that answers your question
Next, let's use the [Exo.MAST Observability Table](https://catalogs.mast.stsci.edu/eaot) to find the highest signal-to-noise target in transmission that is likely to have methane.  Applying the filter: `Teq < 1000 K` and sorting on `K-Band Transmission SNR` shows that WASP-107b is our best target.

# Transiting exoplanet system example
Let's grab all of WASP-107's relevant system parameters using the [ExoMAST](https://exo.mast.stsci.edu) service lookup in ExoCTK.  There are a lot of parameters, so let's just print the planet name and temperature.

In [ ]:
import os

# Get the orbital parameters
from exoctk.utils import get_target_data
wasp107b_params, url = get_target_data('WASP-107b')
print(wasp107b_params['planet_name'], wasp107b_params['Tp'])

# Observation Planning with the Groups and Integrations Calculator
This section will describe planning an observation, including calculating the correct number of groups and Integrations to enter into the Astronomers Proposal Tool (APT).

## Groups and Integrations

To plan an observation with APT, an observer needs to calculate the groups and integerations required for their observation. Each JWST observation consists of some amount of overhead (time to reset the detector, etc), and a discrete number of integrations, each made up of 'n' frames. For time-series observations, there is always only one frame per group, so each integration also has 'n' groups.  The total observation time is determined by the frame time of your given instrument and mode, the number of groups per integration, and the number of integrations in your exposure.  An observation with too many groups per integration will saturate the detector, but too few groups per integration will lead to inefficient observations (i.e. low duty cycles). 

This part of the demo will teach you how to calculate the groups and integrations of our target, WASP 107b, and send it through APT. 

In [ ]:
# Imports for Groups and Integrations
import astropy.units as u
import json

from exoctk.groups_integrations.groups_integrations import perform_calculation

# Specify path to data file
EXOCTK_DATA = os.environ.get('EXOCTK_DATA')
if EXOCTK_DATA == '':
    GROUPS_INTEGRATIONS_DIR = '/path/to/your/local/copy/integrations_groups_data.json'
else:
    GROUPS_INTEGRATIONS_DIR = GROUPS_INTEGRATIONS_DIR = os.path.join(EXOCTK_DATA, 'groups_integrations/groups_integrations.json')

In [ ]:
# Open the file
with open(GROUPS_INTEGRATIONS_DIR) as f:
    parameter_space = json.load(f)
    
# Print the target acquisition (TA) and science observation modes
modes = [('science', 'sci_sat'), ('TA', 'ta_sat')]
for mode in modes:
    print('\n')
    print('Available modes for {} :'.format(mode[0]))
    print('------------------------')
    inss = list(parameter_space[mode[1]].keys())
    print('Instruments : {}'.format(inss))
    for ins in inss:
        filts = list(parameter_space[mode[1]][ins].keys())
        print('Filters for {} : {}'.format(ins, filts))
        subs = list(parameter_space[mode[1]][ins][filts[0]].keys())
        print('Subarrays for {} : {}'.format(ins, subs))

# Print the available Phoenix models
print('\n')
print('Phoenix model keys :')
print('---------------------')
print(list(parameter_space['ta_sat'][inss[-1]][filts[0]][subs[0]].keys()))

print('\n')
print('Magnitude sampling :')
print('--------------------')
print(parameter_space['mags'])

In [ ]:
# Initialize a dictionary of inputs
# When working with the webform, this is created for you
# This information for WASP 107b is available at : https://exo.mast.stsci.edu/exomast_planet.html?planet=WASP19b
parameters = {}

# Source parameters
parameters['mag'] = wasp107b_params['Kmag'] # Stellar mangnitude
parameters['band'] = 'k' # only K band vega mag for now
parameters['mod'] = 'k5v' # Phoenix model per last section

# Observation specifics 
obs_time = wasp107b_params['transit_duration']*u.d.to('h')*3 + 1 # 3*transit duration, converted to hours, plus an extra hour of time
parameters['obs_time'] = obs_time
parameters['n_group'] = 'optimize' # 'optimize', or positive integer

# Detector setup -- within the modes of the last section
parameters['ins'] = 'niriss'
# For science observation
parameters['filt'] = 'soss'
parameters['subarray'] = 'substrip256'
# And target acquisition
parameters['filt_ta'] = 'f480m'
parameters['subarray_ta'] = 'im'

# Saturation level
parameters['sat_mode'] = 'well' # 'well', for full well fraction, or 'counts' 
parameters['sat_max'] = .80 # < 1 for fullwell, and a positive integer always

# And feed in the data file
parameters['infile'] = GROUPS_INTEGRATIONS_DIR
input_dict = parameters.copy()

In [ ]:
# Bookeeping for new/old parameters
inputs = list(parameters.keys())

# Perform the calculation 
results = perform_calculation(parameters)
for key in results:
    if key in inputs:
        if key == 'infile':
            # hackers
            print('The input of infile was REDACTED!')
        else:
            print('The input of {} was {}.'.format(key, results[key]))
    else:
        print('The result of {} was {}'.format(key, results[key]))

In [ ]:
# So let's make a nice printed summary 
print('The total time for science + TA observation scheme is {}-{} hours.'.format(
    results['t_duration']+results['t_duration_ta_max'], results['t_duration']+results['t_duration_ta_min']))
print('You need {} groups and {} integrations for the science observation.'.format(
    results['n_group'], results['n_int']))
print('You need between {} and {} groups for target acquisition.'.format(
    results['max_ta_groups'], results['min_ta_groups']))
print('We estimate your science observation will reach at most {} counts -- how close were we to your cutoff of {}?'.format(
    results['max_sat_prediction'], results['sat_max']))
print('With this observation scheme, {}% of the observation will be spent acquiring data.'.format(results['obs_eff']*100))

# Visibility and Contamination Overlap
Use ExoCTK’s Visibility and Contamination Overlap tool to determine when the target can be observed and at what position angles.

Let's do this part online at https://exoctk.stsci.edu/contam_visibility.

WASP-107 is visible twice per year for about 7 weeks each.

There are two ranges of position angle for which WASP-107 can be observed with NIRISS/SOSS: 99-121 degrees and 285-307 degrees.  Fortunately, there are no nearby stars that could contaminate the spectrum at these position angles.

# Phase Constraint
Use ExoCTK’s Phase Constraint tool to calculate the range of acceptable start times (in units of orbital phase)

In [ ]:
from exoctk.phase_constraint_overlap.phase_constraint_overlap import phase_overlap_constraint
minphase,maxphase = phase_overlap_constraint("WASP-107b", period=None, t0=None, obs_duration=obs_time, window_size=1.)

print(minphase,maxphase)

## Now we can put this data into APT.
Please open your APT application now ...

# Generate Limb Darkening Coefficients
To generate limb darkening coefficients for our host star, we can use the `exoctk.limb_darkening` tool. We can create an `LDC` object using a stellar intensity grid from the Phoenix ACES or Kurucz ATLAS9 models (provided in the `EXOCTK_DATA` zip file).

In [ ]:
from exoctk.limb_darkening.limb_darkening_fit import LDC
from exoctk.modelgrid import ModelGrid
from exoctk.utils import MODELGRID_DIR

atlas9 = ModelGrid(os.path.join(MODELGRID_DIR, 'ATLAS9'), resolution=700)
ld = LDC(atlas9)

Next, we want to generate our limb darkening coefficients using a particular grism throughput using the `svo_filters` package. For this example, we'll use the GR700XD grism to simulate some NIRISS SOSS mode observations of our target, split into 15 equal wavelength bins.

In [ ]:
from svo_filters import Filter
gr700xd = Filter('NIRISS.GR700XD.1', n_bins=15)
gr700xd.plot()

Finally, we can calculate our limb darkening coefficients by passing the host star effective temperature, surface gravity, and metalicity to the `calculate` method. We will also provide the limb darkening profile name and the bandpass we wish to consider:

In [ ]:
teff = wasp107b_params['Teff']
logg = wasp107b_params['stellar_gravity']
feh = wasp107b_params['Fe/H']
print(teff, logg, feh)
ld.calculate(teff, logg, feh, 'quadratic', bandpass=gr700xd)
ld.results.pprint()